In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pandas as pd

In [0]:
catalog_name = "ecommerce"

Removing Spaces from **brand_name** column

In [0]:
from pyspark.sql.functions import trim, col

df_clean = df_bronze.withColumn(
    "brand_name", trim(col("brand_name"))
)

display(df_clean)

brand name should not have special character so replacing it with sapce

In [0]:
df_clean = df_clean.withColumn(
    "brand_id", regexp_replace(col("brand_id"),r'[^0-9-A-Z-a-z]','')
)
# df_clean.limit(100).display()

In [0]:
df_clean.show()


In [0]:
df_clean.select("category_code").distinct().display()


In [0]:
df_clean.show()


In [0]:
anomolies  = {
    "BOOKS":"BKS",
    "GROCERY":"GRC",
    "TOYS":"TOY",
}

In [0]:

df_clean = df_clean.replace(
    to_replace=anomolies, 
    subset=["category_code"]
)

In [0]:
df_clean.select("category_code").distinct().display()


In [0]:
df_clean.display()

In [0]:
df_clean.count()

In [0]:
df_clean.select("brand_id").distinct().count()

In [0]:
df_silver = df_clean\
    .write\
    .mode("overwrite")\
    .option("overwriteSchema", "true")\
    .format("delta")\
    .saveAsTable(f"{catalog_name}.silver.silver_brand")